In [ ]:
# 📦 Installation und 📈 Update des Randstad-Aktienkurses in Smartsheet
!pip install yfinance smartsheet-python-sdk

import yfinance as yf
import smartsheet

# 🔐 Deine Konfiguration
API_KEY = '8C55AWhyXBHDvwpGMRnyogYadTFEv1xzGm37L'   # <-- Dein echter API Token
SHEET_ID = 3347073131827076                        # <-- Deine Sheet-ID
TICKER = 'RAND.AS'                                 # Randstad Ticker bei Yahoo Finance
SPALTENNAME = 'Aktienkurs'                         # Gesuchte Spalte im Sheet

# 📈 1. Aktuellen Kurs abrufen
kurs = yf.Ticker(TICKER).info['currentPrice']
print(f"Aktueller Randstad-Kurs: {kurs} EUR")

# 🔗 2. Smartsheet-Verbindung
client = smartsheet.Smartsheet(API_KEY)
sheet = client.Sheets.get_sheet(SHEET_ID)

# 🧱 3. Spalten-ID finden
col_id = None
for col in sheet.columns:
    print(f"Gefunden: '{col.title}' (ID: {col.id})")  # ← Debug-Ausgabe
    if col.title.strip().lower() == SPALTENNAME.strip().lower():
        col_id = col.id
        break

# ❗Fehlermeldung, wenn Spalte nicht gefunden wurde
if not col_id:
    raise ValueError(f"Spalte '{SPALTENNAME}' nicht gefunden. Bitte prüfe den tatsächlichen Spaltennamen.")

from datetime import datetime

# ✏️ 4. Erste Zeile aktualisieren
row_id = sheet.rows[0].id

# Datum-Spalten-ID suchen
datum_col_id = None
for col in sheet.columns:
    if col.title.strip().lower() == 'datum':
        datum_col_id = col.id

# Kurs-Zelle
kurs_cell = smartsheet.models.Cell()
kurs_cell.column_id = col_id
kurs_cell.value = kurs
kurs_cell.strict = False

# Datum-Zelle
datum_cell = smartsheet.models.Cell()
datum_cell.column_id = datum_col_id
datum_cell.value = datetime.now().strftime('%Y-%m-%d')
datum_cell.strict = False

# Zeile aktualisieren
row = smartsheet.models.Row()
row.id = row_id
row.cells.extend([kurs_cell, datum_cell])

client.Sheets.update_rows(SHEET_ID, [row])
print("✅ Kurs und Datum erfolgreich in Smartsheet aktualisiert.")
